In [ ]:
import sys, os, pathlib
sys.path.append("/root/shared/gitrepos/smart-comp-sci/utils")
sys.path.append("/root/shared/gitrepos/smart-comp-sci/mito-example")
import smart_analysis
from mito_ode import mito_ode_calc
from matplotlib import pyplot as plt
import numpy as np
plt.style.use(str(pathlib.Path.cwd() / ".." / "utils" / "smart_plots.mplstyle"))
cur_dir = pathlib.Path.cwd() / ".." / "analysis_data" / "mito"

Analyze results from mitochondrion simulations. (adjust paths as necessary)

In [ ]:
# files_dir = "/root/scratch/smart-comp-sci-data/mito/results_fixedmodel_coarse"
files_dir = str(pathlib.Path.cwd() / ".." / "analysis_data" / "mito" / "mito_results")
npy_dir = pathlib.Path.cwd() / ".." / "analysis_data" / "mito" / "mito_npy_files"
npy_dir.mkdir(exist_ok=True)
test_folders = os.listdir(files_dir)
condition_str = []
for i in range(len(test_folders)):
    results_folder = f"{files_dir}/{test_folders[i]}"
    if "sepCristae" in test_folders[i]:
        mesh_file = str(pathlib.Path.cwd() / ".." / "analysis_data" / "mito_results" / "mito_mesh_sepCristaeRedo.h5")
    else:
        mesh_file = str(pathlib.Path.cwd() / ".." / "analysis_data" / "mito_results" / "mito_mesh.h5")

    tVec, results_all = smart_analysis.analyze_all(
        mesh_file=mesh_file, results_path=results_folder, 
        display=False, ind_files=True)
    results_all.insert(0, tVec) # add time as first element in list
    max_length = len(tVec)
    for j in range(len(results_all)):
        if len(results_all[j]) > max_length:
            max_length = len(results_all[j])
    for j in range(len(results_all)):
        num_zeros = max_length - len(results_all[j])
        for k in range(num_zeros):
            results_all[j].append(0)
    np.save(npy_dir / f"{test_folders[i]}_results.npy", results_all)

Generate plots of ATP concentration in matrix vs. inner membrane space vs. cytosol (Figure 5). Separate plots for numerical testing of well-mixed case (Figure 6).

In [ ]:
matrix_vol = 0.01646550900025656
om_area = 0.6320190057300417
ims_vol = 0.020313914966362288
im0_area = 0.5255854703788347
cristae_area = 1.0196892371671493
cyto_vol = 0.306
geoParam = [matrix_vol, ims_vol, cyto_vol]

var_names_all = ["ADP_matrix", "ATP_matrix", "ATP_cyto", "ATP_IMS", "L", "DL", "LD", 
                 "LT", "TL", "TLD", "DLD", "DLDp", "TLT", "TLTp", "E_IMS", "E_mat", 
                 "H3EStar_mat", "H3ES_mat", "H3E_mat", "ADP_IMS"]
var_names_saved = ["DLD", "DLT", "DL", "ADP_matrix", "E_IMS", "E_mat", "H3EStar_mat", "H3ES_mat",
                   "H3E_mat", "LD", "LT", "L", "TLD", "TL", "ATP_IMS", "ATP_matrix", "ATP_cyto"]
npyfiles = ["uniform_results.npy", "cristaeLoc_results.npy", "D150_results.npy"] 
Tcytofiles = ["T_cyto_uniform.txt", "T_cyto_cristaeLoc.txt", "T_cyto_D150.txt"]
plot_names = ["ATP_cyto", "ATP_IMS", "ADP_matrix"]
conversions = [cyto_vol * 6.02e5, ims_vol * 6.02e5, matrix_vol * 6.02e5]
for k in range(len(plot_names)):
    plt.figure()
    plot_name = plot_names[k]
    conversion = conversions[k]
    ode_idx = var_names_all.index(plot_name)
    results_idx = var_names_saved.index(plot_name) + 1 # add one because time is first el
    t_ode, ode_results = mito_ode_calc([0, 0.5], geoParam, 10.0)
    plt.plot(1000*t_ode, ode_results[:,ode_idx]/conversion, label=f"ODE", linestyle='dashed')
    for i in range(len(npyfiles)):
        if i == 2:
            plt.figure()
            plt.plot(1000*t_ode, ode_results[:,ode_idx]/conversion, label=f"ODE", linestyle='dashed')
        if plot_name == "ATP_cyto":
            file_cur = f"{cur_dir}/mito_Tcyto_files/{Tcytofiles[i]}"
            results_cur = np.loadtxt(file_cur)
            results_vals = results_cur[:,1]
            results_time = results_cur[:,0]
            plt.plot(1000*results_time, results_vals,label=f"{npyfiles[i][0:-4]}")
        else:
            file_cur = f"{cur_dir}/mito_npy_files/{npyfiles[i]}"
            results_cur = np.load(file_cur)
            results_vals = results_cur[results_idx]
            results_time = results_cur[0]
            plt.plot(1000*results_time, results_vals,label=f"{npyfiles[i][0:-4]}")
        if i == 1:
            plt.legend()
            plt.ylabel(f"{plot_name} (mM)")
            plt.xlabel("Time (ms)")
            plt.xlim([0, 100])
            if plot_name == "ATP_cyto":
                plt.ylim([.308, .33])   
            plt.savefig(f"{cur_dir}/mito_plots/{plot_name}.pdf", format="pdf")
        elif i == 2:
            plt.legend()
            plt.ylabel(f"{plot_name} (mM)")
            plt.xlabel("Time (ms)")
            plt.xlim([0, 50])
            if plot_name == "ATP_cyto":
                plt.ylim([.308, .33])   
            plt.savefig(f"{cur_dir}/mito_plots/{plot_name}-fastdiffusion.pdf", format="pdf")
            plt.figure(figsize=(3,2))
            tInterp = results_time
            odeVals = ode_results[:,ode_idx]/conversion
            odeValInterp = np.interp(tInterp, t_ode, odeVals)
            percent_error = 100*(results_vals-odeValInterp)/odeValInterp
            plt.plot(1000*tInterp, percent_error, label="Percent error")
            plt.ylabel(f"{plot_name} error (%)")
            plt.xlabel("Time (ms)")
            plt.xlim([0, 100])
            # plt.ylim([0, 0.5])
            plt.savefig(f"{cur_dir}/mito_plots/{plot_name}-fastdiffusion-percenterror.pdf", format="pdf")

In [ ]:
var_names_all = ["ADP_matrix", "ATP_matrix", "ATP_cyto", "ATP_IMS", "L", "DL", "LD", 
                 "LT", "TL", "TLD", "DLD", "DLDp", "TLT", "TLTp", "E_IMS", "E_mat", 
                 "H3EStar_mat", "H3ES_mat", "H3E_mat", "ADP_IMS"]
matrix_vol = 0.01646550900025656
om_area = 0.6320190057300417
ims_vol = 0.020313914966362288
im0_area = 0.5255854703788347
cristae_area = 1.0196892371671493
cyto_vol = 0.306
geoParam = [matrix_vol, ims_vol, cyto_vol]
t_ode, ode_results = mito_ode_calc([0, 0.5], geoParam, 10.0)
plt.plot(1000*t_ode, ode_results[:,3], label=f"ODE", linestyle='dashed')